## Homework 6

**Submitted by : Tanvi Arora**   
**Section     : DS 7337 Natural Language Processing - 401**

<a id="top"></a>
### Contents

* <a href="#functions">Function Definitions</a>
* <a href="#titlesimilarity">1 - Book Title Similarity</a>
* <a href="#booksearch">1.a - Book Search on Amazon</a>
* <a href="#pairwisecosinesimilarity">1.b - Pairwise Cosine Similarity</a>
* <a href="#similartitles">1.c - Get titles that are similar to each other</a>
* <a href="#dissimilartitles">1.c - Get titles that are dissimilar to each other</a>
* <a href="#googlesearch">2 - Book Search on Google</a>
* <a href="#googlesearchresults">2.a Search text book on Google</a>
* <a href="#textsimilarity">2.b Resulted Text similarity with search string</a>
* <a href="#textsimilarityanalysis">2.c Analysis on Resulted Text similarity with search string</a>

In [1]:
import platform
print(platform.platform())

import os
print ("environment",os.environ['CONDA_DEFAULT_ENV'])

import sys
print("Python",sys.version)

## for visualizations
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_rows',10)
import re

import numpy as np

## ignore/suppress warnings
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
warnings.simplefilter('ignore', FutureWarning)



Darwin-18.6.0-x86_64-i386-64bit
environment base
Python 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


<a id="functions"></a>
<a href="#top">Back to Top</a>

### Function Definitions

In [2]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
    

## This function returns pairwise cosine similarity matrix of the list
def get_cosine_sim(b_list): 
    vectors = get_vectors(b_list)
    return cosine_similarity(vectors)


## This function applies CountVectorizer to the list and returns a vector matrix    
def get_vectors(b_list):
    vectorizer = CountVectorizer()
    vect = vectorizer.fit_transform(b_list)
    return vect.toarray()

## get maximum and minimum similarities of each book with comparison to other books
## from the pairwise cosine similarity .Function can be used for any list of similarity matrix
def get_max_min(booklist_sim):
    bk_max=[0 for x in range(len(booklist_sim))]
    bk_min=[0 for x in range(len(booklist_sim))]
    bk_mask=np.ma.array(booklist_sim, mask = False)
    for i in range(len(bk_mask)):
        bk_mask[i].mask[i] = True
        bk_max[i]=max(bk_mask[i].compressed())
        bk_min[i]=min(bk_mask[i].compressed())
    return bk_max,bk_min

## This function returns the book with maximum similarity for all books
## pairwise similarity of a book with itself,i.e. diagonal in the matrix 
## is ignored
def get_max_similar_books(max_mt, booklist_sim, books):
    bk_mask_max=np.ma.array(max_mt, mask = False)
    max_sim=max(bk_mask_max)
    for i in range(len(books)):
    
        if (bk_mask_max[i]==max_sim):
            print("Title of Book : ",books[i], "  is very much similar to Title of the book : ")
            for j in range(len(booklist_sim)):
                if (i != j):
                    if (booklist_sim[i][j] == max_sim):
                        print(books[j])
                        print("The ranks of the book are ",i, " and " ,j)
                        bk_mask_max.mask[j] = True
            print("---------------------------------")
                    

## This function returns the books with minimum similarity for all books
def get_min_similar_books(min_mt, booklist_sim, books):
    min_sim=min(min_mt)
    for i in range(len(books)):
        bk_mask_min=np.ma.array(min_mt, mask = False)
        if (bk_mask_min[i]==min_sim):
            print("Title of Book : ",books[i], "with rank as ",i," is very much dissimilar to below list of books : ")
            print("")
            for j in range(len(booklist_sim)):
                if (booklist_sim[i][j] == min_sim):
                    print(books[j])
                    print("Rank of this book is ",j)
                    bk_mask_min.mask[j] = True
            print("---------------------------------")    
            

# This function returns book with maximum similarity of a particular book in the list
def get_max_similar_books_id(max_sim, booklist_sim, books,index):
    i=index
    print("Title of Book : ",books[i], "  is very much similar to Title of the book : ")
    for j in range(len(booklist_sim)):
        if (i != j):
            if (booklist_sim[j] == max_sim):
                print(books[j])
                print("The ranks of the book are ",i, " and " ,j)
    print("---------------------------------")
                    
# This function returns book with minimum similarity of a particular book in the list
def get_min_similar_books_id(min_sim, booklist_sim, books,index):
    i=index
    print("Title of Book : ",books[i], "with rank as ",i," is very much dissimilar to below list of books : ")
    print("")
    for j in range(len(booklist_sim)):
        if (booklist_sim[j] == min_sim):
            print(books[j])
            print("Rank of this book is ",j)
    print("---------------------------------") 

<a id="titlesimilarity"></a>
<a href="#top">Back to Top</a>

### Book Title Similarity

<a id="booksearch"></a>
<a href="#top">Back to Top</a>

#### Search for a book on Amazon

Amazon search : https://www.amazon.com/s?k=Autobiography+of+M+K+Gandhi&ref=nb_sb_noss  
book searched : Autobiography of M K Gandhi


In [3]:
booklist=["An Autobiography or The Story of My Experiments with Truth Kindle Edition",
          "An Autobiography: Or, The Story of My Experiments with Truth Kindle Edition",
          "An Autobiography or The Story of My Experiments with Truth Kindle Edition",
         "The Essential Gandhi: An Anthology of His Writings on His Life, Work, and Ideas Paperback – November 12, 2002",
         "SUMMARY OF An Autobiography | M.K. Gandhi | Kindle Ebooks| Highlights and Key Concepts | Save Money and Time Reading Summaries Kindle Edition",
         "An Autobiography: The Story of My Experiments with Truth 2nd Edition",
         "An Autobiography: The Story of my Experiments with Truth Paperback – September 14, 2017",
         "The Story Of My Experiments With Truth: An Autobiography of Mahatma Gandhi Kindle Edition",
         "An Autobiography or The Story of My Experiments with Truth: A Critical Edition Hardcover – March 20, 2018",
         "M.K. Gandhi an Autobiography:The Story of My Experiments with Truth Hardcover – April 4, 2018",
         "Non-Violent Resistance (Satyagraha) Paperback – May 29, 2001",
         "M. K. Gandhi An Autobiography or the Story of My Experiments with Truth Paperback – 1997",
         "An Autobiography: 150th Anniversary Edition Paperback – September 12, 2019",
         "The Autobiography of M.K.Gandhi (Chinese Edition) (Chinese) Hardcover – October 1, 2015",
         "My Experiments with Truth: An Autobiography Kindle Edition",
         "Satya Ke Sath Mere Prayog [My Experiment with Truth]: Meri Atmakatha [My Autobiography]   Audible Audiobook – Unabridged",
         "Mahatma Gandhi: A Life From Beginning to End Kindle Edition",
         "(An Autobiography or the story of my experiments with truth) Satya ke Prayog Athava Atmakatha (Hindi Edition) (Hindi) Paperback – January 1, 2012",
         "M.K. Gandhi, Attorney at Law: The Man before the Mahatma First Edition",
         "Autobiography: The Story of My Experiments with Truth by Mohandas Karamchand (Mahatma) Gandhi (2008-10-01) Hardcover – 1796",
         "The Story of My Experiments with Truth: Autobiography of M K Gandhi",
         "An Autobiography Paperback – Import, 1992",
         "M.K. GANDHI, AN AUTOBIOGRAPHY, FOLIO SOCIETY COLLECTOR'S EDITION Hardcover – 2010",
         "M.k. Gandhi : An Autobiography Paperback – 2015",
         "Satyana Prayogo Athva Atmakatha (Gujarati Edition) Kindle Edition"]

In [4]:
len(booklist)

25

A list of top 25 books from the Amazon search has been created

<a id="pairwisecosinesimilarity"></a>
<a href="#top">Back to Top</a>

#### Get pairwise cosine similarity for all books

In [5]:
booklist_cos_sim=get_cosine_sim(booklist)

In [6]:
print("type :", type(booklist_cos_sim))
print("shape :", booklist_cos_sim.shape)

type : <class 'numpy.ndarray'>
shape : (25, 25)


**We have 25 books and the cosine similarity function has created a 2D array( matrix) of 25X25**  
Next we will fetch maximum and minimum similarity value of each book . For maximum similarity, similarity value of the book with itself is ignored as it will always be max value i.e. 1.0

In [7]:
bk_cos_max,bk_cos_min=get_max_min(booklist_cos_sim)

<a id="similartitles"></a>
<a href="#top">Back to Top</a>

#### Get titles that are most similar to each other

In [8]:
max(bk_cos_max)

1.0000000000000002

In [9]:
get_max_similar_books(bk_cos_max,booklist_cos_sim,booklist)

Title of Book :  An Autobiography or The Story of My Experiments with Truth Kindle Edition   is very much similar to Title of the book : 
An Autobiography: Or, The Story of My Experiments with Truth Kindle Edition
The ranks of the book are  0  and  1
An Autobiography or The Story of My Experiments with Truth Kindle Edition
The ranks of the book are  0  and  2
---------------------------------


We have a book title matching another book title with a cosine similarity of 1.0 , since we have ignored similarity values of a book with itself this has to be another book. But when we ran the function to get the book title names, it was exactly the same book.Top 3 books , rank 0,1,2  looks to be the same books.  
Below is the screenshot of the actual books listed by Amazon.  
1. The titles match but the covers do not and one of them has a joint author too.  
2. Also the first book has additional special characters - ":" ( colon) and "," ( comma ) , and capitalized "O" in "Or" but the cosine similarity algorithm has matched the books equally.

![top3](top3_Gandhi_Autobigraphy_books.png)

<a id="dissimilartitles"></a>
<a href="#top">Back to Top</a>
#### Get titles that are most dissimilar

In [10]:
min(bk_cos_min)

0.0

In [11]:
get_min_similar_books(bk_cos_min,booklist_cos_sim,booklist)

Title of Book :  An Autobiography or The Story of My Experiments with Truth Kindle Edition with rank as  0  is very much dissimilar to below list of books : 

Non-Violent Resistance (Satyagraha) Paperback – May 29, 2001
Rank of this book is  10
---------------------------------
Title of Book :  An Autobiography: Or, The Story of My Experiments with Truth Kindle Edition with rank as  1  is very much dissimilar to below list of books : 

Non-Violent Resistance (Satyagraha) Paperback – May 29, 2001
Rank of this book is  10
---------------------------------
Title of Book :  An Autobiography or The Story of My Experiments with Truth Kindle Edition with rank as  2  is very much dissimilar to below list of books : 

Non-Violent Resistance (Satyagraha) Paperback – May 29, 2001
Rank of this book is  10
---------------------------------
Title of Book :  The Essential Gandhi: An Anthology of His Writings on His Life, Work, and Ideas Paperback – November 12, 2002 with rank as  3  is very much diss

Smallest cosine similarity minimum value is 0. We have many books with 0.0 similarity value as you can see form the output above.
Out of the list there are 2 books that had maximum dissimilarities -
1. Non-Violent Resistance (Satyagraha) Paperback – May 29, 2001 , which was ranked "10" had dissimilarities with 17 out of the 25 books.
2. Satyana Prayogo Athva Atmakatha (Gujarati Edition) Kindle Edition, which was ranked 24 i.e. last of our list had dissimilarities with 9 out of 25 books . The title of this book is in an Indian language "Gujrati" which essentially matches the topic of our search if translated in English. It still showed up at #25 , which could mean Amazon utilizes some of language translations for the match. 

**In general Amazon returns a wide variety of search and so if we look at pairwise text similarity, there are many that do not match in the top 25 books. It is the title, but if you read the titles, except for a couple, most of the books returned by search results intend to support the search string. The couple ones that may seem to be not related to his autobiography can still be called his biographies as they are related to Mahatma Gandhi's life work.**

<a id="googlesearch"></a>
<a href="#top">Back to Top</a>

### Book Search on Google

Next we pick up a title of book and search it on Google.  Below is the screenshot of the 1st capsule and 20th capsule ignoring any sponsored links.  

<a id="googlesearchresults"></a>
<a href="#top">Back to Top</a>
#### Search text book on Google

**Capsule# 1**

<img src="./capsule_1.png">

**Capsule# 20**

<img src="./capsule_20.png">

<a id="textsimilarity"></a>
<a href="#top">Back to Top</a>
#### Text similarity on search string and search results on google

Grabbing the search text ( Title of the book ) and preview text from capsule1 and capsule 20, we will apply cosine similarity function to analyze which search result matched better with the search text.

In [12]:
search_str=["An Autobiography or The Story of My Experiments"]
capsule_list=["The Story of My Experiments with Truth is the autobiography of Mohandas K. Gandhi, covering his life from early childhood through to 1921. It was written in ...",
       "An autobiography : the story of my experiments with truth. [Gandhi, Mahatma] -- In his classic autobiography Gandhi recounts the story of his life and how he ..."]
b_list=[s for s in search_str]
for c in capsule_list:
    b_list.append(c)
b_list
print(len(b_list))
#searchstr_cos_sim=get_cosine_sim_with_str(booklist,search_str)

3


In [13]:
searchstr_cos_sim=get_cosine_sim(b_list)

In [14]:
searchstr_cos_sim=get_cosine_sim(b_list)
print("type :",type(searchstr_cos_sim))
print("shape :",searchstr_cos_sim.shape)

type : <class 'numpy.ndarray'>
shape : (3, 3)


Our list of strings/document is 3 and cosine similarity function has returned a 3X3 matrix for pairwise cosine similarity. Our search string was at index "0", so we will analyze pairwise similarity for index 0.

In [15]:
searchstr_max,searchstr_min=get_max_min(searchstr_cos_sim)
max_match_wt_searchstr=searchstr_max[0]
min_match_wt_searchstr=searchstr_min[0]

In [16]:
get_max_similar_books_id(max_match_wt_searchstr, searchstr_cos_sim[0], b_list,0)


Title of Book :  An Autobiography or The Story of My Experiments   is very much similar to Title of the book : 
An autobiography : the story of my experiments with truth. [Gandhi, Mahatma] -- In his classic autobiography Gandhi recounts the story of his life and how he ...
The ranks of the book are  0  and  2
---------------------------------


In [17]:
get_min_similar_books_id(min_match_wt_searchstr, searchstr_cos_sim[0], b_list,0)

Title of Book :  An Autobiography or The Story of My Experiments with rank as  0  is very much dissimilar to below list of books : 

The Story of My Experiments with Truth is the autobiography of Mohandas K. Gandhi, covering his life from early childhood through to 1921. It was written in ...
Rank of this book is  1
---------------------------------


In [18]:
print("maximum similarity", searchstr_cos_sim[0][2])

maximum similarity 0.639362013091006


Text in capsule 20 has the highest similarity measure with the search string with the similarity of 63.9%.

<a id="textsimilarityanalysis"></a>
<a href="#top">Back to Top</a>
#### Analysis on Resulted Text similarity with search string

Same Text similarity function when applied shows Google search seems to use a different algorithm. Out most similar match for the search string is in capsule 20 . So why is that so ?
Google uses a series of algorithms to decide on ranks. There are many factors that are looked into and evalauted with different weights.  
Besides the meaning of the query and relevance of webpages ,i.e. finding similar search text in the webpage, it also looks for quality of content which is determined by pagerank.This selects pages that demonstrate expertise, authoritativeness and trustworthiness on a given topic.  
While this might be the main criteria that may cause the difference in similarity of the returned results, google also looks for any unusable links and returns results based on context, for eg. location or language settings.
